## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-11-20-46-01 +0000,nyt,"Thousands Return to Northern Gaza, Hopeful, bu...",https://www.nytimes.com/2025/10/11/world/middl...
1,2025-10-11-20-44-50 +0000,nypost,Israeli terror victim recounts attack that too...,https://nypost.com/2025/10/11/world-news/israe...
2,2025-10-11-20-39-04 +0000,nyt,Trump Says He Will Pay Troops Despite Governme...,https://www.nytimes.com/2025/10/11/us/politics...
3,2025-10-11-20-36-19 +0000,nyt,"At the Center of Letitia James’s Indictment, a...",https://www.nytimes.com/2025/10/11/us/politics...
4,2025-10-11-20-36-00 +0000,wsj,Trump said funds have been identified to pay m...,https://www.wsj.com/politics/policy/trump-says...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2315/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
20,trump,45
3,gaza,14
26,shutdown,14
131,new,12
267,china,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
37,2025-10-11-18-12-00 +0000,nypost,President Trump will make sure military troops...,https://nypost.com/2025/10/11/us-news/presiden...,107
43,2025-10-11-17-17-44 +0000,nypost,Putin chides Nobel winners who ‘did nothing fo...,https://nypost.com/2025/10/11/world-news/putin...,105
212,2025-10-10-22-34-17 +0000,nypost,Trump gives new details on Mideast trip as Ham...,https://nypost.com/2025/10/10/us-news/trump-gi...,99
4,2025-10-11-20-36-00 +0000,wsj,Trump said funds have been identified to pay m...,https://www.wsj.com/politics/policy/trump-says...,98
2,2025-10-11-20-39-04 +0000,nyt,Trump Says He Will Pay Troops Despite Governme...,https://www.nytimes.com/2025/10/11/us/politics...,93


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
37,107,2025-10-11-18-12-00 +0000,nypost,President Trump will make sure military troops...,https://nypost.com/2025/10/11/us-news/presiden...
43,60,2025-10-11-17-17-44 +0000,nypost,Putin chides Nobel winners who ‘did nothing fo...,https://nypost.com/2025/10/11/world-news/putin...
236,45,2025-10-10-21-13-00 +0000,wsj,"House Speaker Mike Johnson (R., La.) received ...",https://www.wsj.com/politics/policy/mike-johns...
35,43,2025-10-11-18-19-42 +0000,cbc,Tennessee authorities say no survivors found i...,https://www.cbc.ca/news/world/tennessee-explos...
114,43,2025-10-11-10-00-00 +0000,latimes,Palisades fire suspect placed gun in stuffed a...,https://www.latimes.com/california/story/2025-...
144,38,2025-10-11-06-44-04 +0000,nyt,North Korea Flaunts New Missiles ​in Parade Wi...,https://www.nytimes.com/2025/10/11/world/asia/...
164,33,2025-10-11-02-31-03 +0000,nypost,NY AG Letitia James admitted Virginia home at ...,https://nypost.com/2025/10/10/us-news/ny-ag-le...
76,31,2025-10-11-14-33-19 +0000,nypost,"4 killed, at least 20 hurt as ‘hundreds of sho...",https://nypost.com/2025/10/11/us-news/at-least...
122,31,2025-10-11-09-40-22 +0000,nypost,Kim Jong Un shows off ‘most powerful’ missile ...,https://nypost.com/2025/10/11/world-news/kim-j...
200,29,2025-10-10-23-21-00 +0000,missionlocal,Sutter tiene grandes planes de atención médica...,https://missionlocal.org/2025/10/sutter-salud-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
